In [18]:
import redis
import json
import time

In [26]:
pool = redis.ConnectionPool(host='127.0.0.1', port=6379, db=0)
r = redis.Redis(connection_pool=pool)

In [27]:
data = None

with open('large-file.json') as file:
    data = json.load(file)

len(data)

11351

In [28]:
def measure_time(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start} seconds to run.")
        return result
    return wrapper

In [91]:
@measure_time
def save_as_string():
    r.set('as_string', json.dumps(data))

@measure_time
def save_as_hash():
    r.hset('as_hash', 'as_hash', json.dumps(data))

@measure_time
def save_as_zset():
    for key, value in enumerate(data):
        r.zadd('as_zset', {json.dumps(value): key})

@measure_time
def save_as_json_list():
    for value in data:
        r.rpush('as_json_list', json.dumps(value))


In [92]:
save_as_string()
save_as_hash()
save_as_zset()
save_as_json_list()

save_as_string took 0.1463298797607422 seconds to run.
save_as_hash took 0.1550300121307373 seconds to run.
save_as_zset took 0.488783597946167 seconds to run.
save_as_json_list took 0.5516571998596191 seconds to run.


In [93]:
@measure_time
def read_as_string():
    r.get('as_string')

@measure_time
def read_as_hash():
    r.hget('as_hash', 'as_hash')

@measure_time
def read_as_zset():
    r.zrange('as_zset', 0, -1)

@measure_time
def read_as_json_list():
    r.lrange('as_json_list', 0, -1)

In [94]:
read_as_string()
read_as_hash()
read_as_zset()
read_as_json_list()

read_as_string took 0.025226831436157227 seconds to run.
read_as_hash took 0.017345666885375977 seconds to run.
read_as_zset took 0.055353403091430664 seconds to run.
read_as_json_list took 0.29386091232299805 seconds to run.


#### Запустим теперь кластер из 6 нод (см. файл create_cluster.ipynb)

In [112]:
pool = redis.ConnectionPool(host='127.0.0.1', port=7001, db=0)
r = redis.Redis(connection_pool=pool)

read_as_string()
read_as_hash()
read_as_zset()
read_as_json_list()

read_as_string()
read_as_hash()
read_as_zset()
read_as_json_list()

read_as_string took 0.0008826255798339844 seconds to run.
read_as_hash took 0.00011157989501953125 seconds to run.
read_as_zset took 5.698204040527344e-05 seconds to run.
read_as_json_list took 3.743171691894531e-05 seconds to run.
read_as_string took 2.7894973754882812e-05 seconds to run.
read_as_hash took 2.6941299438476562e-05 seconds to run.
read_as_zset took 3.314018249511719e-05 seconds to run.
read_as_json_list took 7.796287536621094e-05 seconds to run.


#### Результаты производительности кода выше разные на одноузловом Redis и Redis Cluster из-за их архитектурных различий. Причиной того, что код работает быстрее на Redis, является то, что запросы из задания выполняются последовательно, то есть мы не экономим на горизонтальном масштабировании. Ведь время не меньше, чем максимум по всем нодам, а еще их запросы синхронизации также дают задержку.